# Excel to JSON
The following code converts the AISC shapes database excel file to JSON.

The original file contains two sections per row. The left has
imperial, and the right metric units, but they do not correspond to
the exact same section (the metric section dimensions are close to
the imperial size but rounded up, they are not merely
converted). The following code reads the imperial sections only.

To decrease the lookup time for the subsequent analysis, we only
extract data for specific section types.

In [ ]:
import json
from pathlib import Path

import pandas as pd

In [ ]:
contents = pd.read_excel(
    'aisc-shapes-database-v15.0.xlsx', sheet_name='Database v15.0', usecols='A:CF'
)

In [ ]:
# instantiate an empty list to store the dictionaries
sections = {}
for i in range(len(contents)):  # for each row
    # turn row into a dictionary
    dct = dict(contents.loc[i])

    # we only want data for specific section types
    if dct['Type'] not in {'W', 'HSS'}:
        continue

    # filter out key-value pairs
    # where value = '-'
    # and redundant keys
    new_dct = {}
    for key, value in dct.items():
        if value != '–':  # noqa: RUF001
            new_dct[key] = value  # noqa: PERF403
    # add it to the list
    sections[dct['AISC_Manual_Label']] = new_dct

In [ ]:
# save to a JSON file
with Path('data/sections.json').open('w', encoding='utf-8') as f:
    f.write(json.dumps(sections))